In [1]:
#%matplotlib inline
import os

import intake
import satsearch
from satsearch import Search

import ast
import pandas as pd
import numpy as np
import xarray as xr

import geopandas as gpd
import rasterio
import rioxarray

import dask
from dask.diagnostics import ProgressBar
#import dask.distributed

#import geoviews as gv
#import hvplot.pandas
from ipywidgets import interact
from IPython.display import display, Image

import hdmedians


In [2]:
env = dict(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR', 
           AWS_NO_SIGN_REQUEST='YES',
           GDAL_MAX_RAW_BLOCK_CACHE_SIZE='250000000',
           GDAL_SWATH_SIZE='250000000',
           VSI_CURL_CACHE_SIZE='250000000')
os.environ.update(env)

In [3]:
bbox = [35.48, -3.24, 35.58, -3.14] # (min lon, min lat, max lon, max lat)
bbox = [129.0000000000000000,-39.0000000000000000, 141.0000000000000000,-26.0000000000000000]
bbox = [138.54207373854032 - 0.01, -34.934374426311145 - 0.01, 138.54207373854032 + 0.01, -34.934374426311145 + 0.01]
#bbox = [138.54207373854032 - 0.5, -34.934374426311145 - 0.5, 138.54207373854032 + 0.5, -34.934374426311145 + 0.5]

dates = '2017-01-01/2020-12-31'

#https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs
URL='https://earth-search.aws.element84.com/v0'
results = Search(url=URL,
                                  collections=['sentinel-s2-l2a-cogs'], # note collection='sentinel-s2-l2a-cogs' doesn't work
                                  datetime=dates,
                                  bbox=bbox,   
                                  #property=["eo:cloud_cover<1"],
                                  #query={'eo:cloud_cover': {'lt': 1}},
                                  query={'eo:cloud_cover': {'eq': 0}, 'platform':{'eq': 'sentinel-2a'}},
                                  sort=['<datetime'])

In [4]:
print('%s items' % results.found())
items = results.items()
# Save this locally for use later
#items.save('my-s2-l2a-cogs-sa-cloudcover1.json')
items.save('my-s2-l2a-cogs-sa-cloudcover0OZLocation.geojson')

40 items


In [5]:
#catalog = intake.open_stac_item_collection(results.items())

In [6]:
import geopandas as gpd
#gf = gpd.read_file('my-s2-l2a-cogs-sa-cloudcover1.geojson')
gf2 = gpd.read_file('my-s2-l2a-cogs-sa-cloudcover0OZLocation.geojson')

In [7]:
gf2.head()

,id,datetime,platform,constellation,gsd,view:off_nadir,proj:epsg,sentinel:utm_zone,sentinel:latitude_band,sentinel:grid_square,sentinel:sequence,sentinel:product_id,sentinel:data_coverage,eo:cloud_cover,sentinel:valid_cloud_cover,created,updated,data_coverage,geometry
0,S2A_54HTG_20201220_0_L2A,2020-12-20T00:56:43+00:00,sentinel-2a,sentinel-2,10,0,32754,54,H,TG,0,S2A_MSIL2A_20201220T004701_N0214_R102_T54HTG_2...,99.80,0,True,2020-12-20T06:18:58.775002+00:00,2020-12-20T06:18:58.775002+00:00,NaN,"POLYGON ((138.87370 -35.31256, 137.70126 -35.2..."
1,S2A_54HTG_20201130_0_L2A,2020-11-30T00:56:45+00:00,sentinel-2a,sentinel-2,10,0,32754,54,H,TG,0,S2A_MSIL2A_20201130T004711_N0214_R102_T54HTG_2...,99.84,0,True,2020-11-30T07:13:47.674000+00:00,2020-11-30T07:13:47.674000+00:00,NaN,"POLYGON ((138.87711 -35.31261, 137.70126 -35.2..."
2,S2A_54HTG_20201117_0_L2A,2020-11-17T00:46:52+00:00,sentinel-2a,sentinel-2,10,0,32754,54,H,TG,0,S2A_MSIL2A_20201117T003711_N0214_R059_T54HTG_2...,61.38,0,True,2020-11-17T05:53:27.379000+00:00,2020-11-17T05:53:27.379000+00:00,NaN,"POLYGON ((138.90758 -35.31315, 138.02743 -35.2..."
3,S2A_54HTG_20200829_0_L2A,2020-08-29T00:46:54+00:00,sentinel-2a,sentinel-2,10,0,32754,54,H,TG,0,S2A_MSIL2A_20200829T003711_N0214_R059_T54HTG_2...,61.31,0,False,2020-08-29T06:53:02.125000+00:00,2020-08-29T06:53:02.125000+00:00,NaN,"POLYGON ((138.90758 -35.31315, 138.02787 -35.2..."
4,S2A_54HTG_20200819_0_L2A,2020-08-19T00:46:54+00:00,sentinel-2a,sentinel-2,10,0,32754,54,H,TG,0,S2A_MSIL2A_20200819T003711_N0214_R059_T54HTG_2...,61.32,0,False,2020-08-20T00:25:47.097000+00:00,2020-08-20T00:25:47.097000+00:00,NaN,"POLYGON ((138.90758 -35.31315, 138.02765 -35.2..."


In [8]:
def get_STAC_items(url, collection, dates, bbox):
    results = Search.search(url=url,
                        collections=[collection], 
                        datetime=dates,
                        bbox=bbox,    
                        query={'eo:cloud_cover': {'eq': 0}, 'platform':{'eq': 'sentinel-2a'}},
                        sortby=['-properties.datetime'])

    items = results.items()
    print(f'Found {len(items)} Items')
    
    return intake.open_stac_item_collection(items)

In [9]:
URL2='https://earth-search.aws.element84.com/v0'
collection2 = 'sentinel-s2-l2a-cogs' # sentinel-2 w/ atmospheric corrections
from satsearch import Search
items_s2 = get_STAC_items(URL2, collection2, dates, bbox)

#query={'eo:cloud_cover': {'eq': 0}, 'platform':{'eq': 'sentinel-2a'}},


Found 40 Items


In [10]:
gf2function = items_s2.to_geopandas()

C:\Users\rscott\AppData\Local\Continuum\anaconda3\envs\pangeo3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [11]:
gf2function.head()

,geometry,datetime,platform,constellation,instruments,gsd,view:off_nadir,proj:epsg,sentinel:utm_zone,sentinel:latitude_band,sentinel:grid_square,sentinel:sequence,sentinel:product_id,sentinel:data_coverage,eo:cloud_cover,sentinel:valid_cloud_cover,created,updated,data_coverage
0,"POLYGON ((138.87370 -35.31256, 137.70126 -35.2...",2020-12-20T00:56:43Z,sentinel-2a,sentinel-2,[msi],10,0,32754,54,H,TG,0,S2A_MSIL2A_20201220T004701_N0214_R102_T54HTG_2...,99.80,0,True,2020-12-20T06:18:58.775Z,2020-12-20T06:18:58.775Z,NaN
1,"POLYGON ((138.87711 -35.31261, 137.70126 -35.2...",2020-11-30T00:56:45Z,sentinel-2a,sentinel-2,[msi],10,0,32754,54,H,TG,0,S2A_MSIL2A_20201130T004711_N0214_R102_T54HTG_2...,99.84,0,True,2020-11-30T07:13:47.674Z,2020-11-30T07:13:47.674Z,NaN
2,"POLYGON ((138.90758 -35.31315, 138.02743 -35.2...",2020-11-17T00:46:52Z,sentinel-2a,sentinel-2,[msi],10,0,32754,54,H,TG,0,S2A_MSIL2A_20201117T003711_N0214_R059_T54HTG_2...,61.38,0,True,2020-11-17T05:53:27.379Z,2020-11-17T05:53:27.379Z,NaN
3,"POLYGON ((138.90758 -35.31315, 138.02787 -35.2...",2020-08-29T00:46:54Z,sentinel-2a,sentinel-2,[msi],10,0,32754,54,H,TG,0,S2A_MSIL2A_20200829T003711_N0214_R059_T54HTG_2...,61.31,0,False,2020-08-29T06:53:02.125Z,2020-08-29T06:53:02.125Z,NaN
4,"POLYGON ((138.90758 -35.31315, 138.02765 -35.2...",2020-08-19T00:46:54Z,sentinel-2a,sentinel-2,[msi],10,0,32754,54,H,TG,0,S2A_MSIL2A_20200819T003711_N0214_R059_T54HTG_2...,61.32,0,False,2020-08-20T00:25:47.097Z,2020-08-20T00:25:47.097Z,NaN


In [13]:
%%time

# First let's construct a large dataset with all the scenes in our search so that we
# have a time dimension
# Loop through geopandas geodataframe (each row is a STAC ITEM)
import dask

@dask.delayed
def stacitem_to_dataset2(item, datetimeuse):
    print(item)
    #stacked = catalog[item.id].stack_bands(bands)
    stacked = item.stack_bands(['B02','B03','B04','B05'])
    #stacked = item.stack_bands(['B02'])
    da = stacked(chunks=dict(band=1, x=2048, y=2048)).to_dask()
    #da = stacked(chunks=dict(band=1, x=5120, y=5120)).to_dask()
    #da['band'] = bands # use common names
    da['band'] = ['blue','green','red','nir'] # use common names
    #da['band'] = ['blue'] # use common names
    da = da.expand_dims(time=[pd.to_datetime(datetimeuse)])
    #da = stacked(chunks=dict(band=1, x=5120, y=5120, time=-1)).to_dask()
    ds = da.to_dataset(dim='band')
    return ds

lazy_datasets = []
#for i,item in gf.iterrows():
#for i,item in gf.head(1).iterrows():   

for index, x in enumerate(items_s2):
    #need to make items_s2
    #need to get datetime from the dataframe gf2 as well
    datetimeuse = gf2function.iloc[index]['datetime']
    itemx = items_s2[x]
    ds = stacitem_to_dataset2(itemx, datetimeuse)
    lazy_datasets.append(ds)

from dask.diagnostics import ProgressBar
with ProgressBar():
    #out = dask.compute(*lazy_datasets)
    
#datasets = dask.compute(*lazy_datasets).mean(dim='time')
    datasets = dask.compute(*lazy_datasets)

[                                        ] | 0% Completed |  0.0s<Intake catalog: S2A_54HTG_20180125_0_L2A>
<Intake catalog: S2A_54HTG_20171017_0_L2A>
<Intake catalog: S2A_54HTG_20200819_0_L2A><Intake catalog: S2A_54HTG_20190301_0_L2A>
<Intake catalog: S2A_54HTG_20201117_0_L2A>

<Intake catalog: S2A_54HTG_20190917_0_L2A>
<Intake catalog: S2A_54HTG_20191216_0_L2A>
<Intake catalog: S2A_54HTG_20190507_1_L2A>
<Intake catalog: S2A_54HTG_20181118_0_L2A>
<Intake catalog: S2A_54HTG_20181231_0_L2A>
<Intake catalog: S2A_54HTG_20180422_0_L2A>
<Intake catalog: S2A_54HTG_20180525_0_L2A>
[                                        ] | 0% Completed | 52.4s<Intake catalog: S2A_54HTG_20170619_0_L2A>
[#                                       ] | 2% Completed | 52.5s<Intake catalog: S2A_54HTG_20201130_0_L2A>
<Intake catalog: S2A_54HTG_20180323_0_L2A>
[###                                     ] | 7% Completed | 52.6s<Intake catalog: S2A_54HTG_20200102_0_L2A>
<Intake catalog: S2A_54HTG_20180714_0_L2A>
[######  

In [14]:
#datasets = dask.compute(*lazy_datasets)
datasets

(<xarray.Dataset>
 Dimensions:  (time: 1, x: 16470, y: 16470)
 Coordinates:
   * time     (time) object 2020-12-20T00:56:43+00:00
   * x        (x) float64 2e+05 2e+05 2e+05 ... 3.098e+05 3.098e+05 3.098e+05
   * y        (y) float64 6.09e+06 6.09e+06 6.09e+06 ... 6.2e+06 6.2e+06 6.2e+06
 Data variables:
     blue     (time, y, x) float64 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
     green    (time, y, x) float64 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
     red      (time, y, x) float64 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
     nir      (time, y, x) float64 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
 Attributes:
     transform:           (10.0, 0.0, 199980.0, 0.0, -10.0, 6200020.0)
     crs:                 +init=epsg:32754
     res:                 (10.0, 10.0)
     is_tiled:            1
     nodatavals:          (0.0,)
     scales:              (1.0,)
     offsets:             (0.0,)
     AREA_OR_POINT:       Area
     OVR_R

In [16]:
DS = xr.concat(datasets, dim='time')

In [17]:
#DS['blue'].quantile(0.5,dim='time')
DS

,Array,Chunk
Bytes,86.80 GB,33.55 MB
Shape,"(40, 16470, 16470)","(1, 2048, 2048)"
Count,119781 Tasks,3240 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,86.80 GB,33.55 MB
Shape,"(40, 16470, 16470)","(1, 2048, 2048)"
Count,119781 Tasks,3240 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


In [19]:
DS.blue

,Array,Chunk
Bytes,86.80 GB,33.55 MB
Shape,"(40, 16470, 16470)","(1, 2048, 2048)"
Count,119781 Tasks,3240 Chunks
Type,float64,numpy.ndarray


In [23]:
def oldmedian(array, dim, keep_attrs=False, skipna=False, **kwargs):
    """ Runs a median on an dask-backed xarray.
    
    This function does not scale!
    It will rechunk along the given dimension, so make sure 
    your other chunk sizes are small enough that it 
    will fit into memory.
    
    :param DataArray array: An xarray.DataArray wrapping a dask array
    :param dim str: The name of the dim in array to calculate the median
    """
    if type(array) is xr.Dataset:
        return array.apply(median, dim=dim, keep_attrs=keep_attrs, **kwargs)
    
    if not hasattr(array.data, 'dask'):
        return array.median(dim, keep_attrs=keep_attrs, **kwargs)
    
    array = array.chunk({dim:-1})
    axis = array.dims.index(dim)
    median_func = np.nanmedian if skipna else np.median
    blocks = dask.array.map_blocks(median_func, array.data, dtype=array.dtype, drop_axis=axis, axis=axis, **kwargs)
    
    new_coords={k: v for k, v in array.coords.items() if k != dim and dim not in v.dims}
    new_dims = tuple(d for d in array.dims if d != dim)
    new_attrs = array.attrs if keep_attrs else None
    
    return xr.DataArray(blocks, coords=new_coords, dims=new_dims, attrs=new_attrs)

#https://github.com/opendatacube/datacube-notebooks/blob/master/utils/data_cube_utilities/dc_mosaic.py
#output = create_hdmedians_multiple_band_mosaic(xarr0)
#del xarr0
def create_default_clean_mask(dataset_in):
    """
    Description:
        Creates a data mask that masks nothing.
    -----
    Inputs:
        dataset_in (xarray.Dataset) - dataset retrieved from the Data Cube.
    Throws:
        ValueError - if dataset_in is an empty xarray.Dataset.
    """
    data_vars = dataset_in.data_vars
    if len(data_vars) != 0:
        first_data_var = next(iter(data_vars))
        clean_mask = np.ones(dataset_in[first_data_var].shape).astype(np.bool)
        return clean_mask
    else:
        raise ValueError('`dataset_in` has no data!')
        
def create_hdmedians_multiple_band_mosaic(dataset_in,
                                          clean_mask=None,
                                          no_data=-9999,
                                          dtype=None,
                                          intermediate_product=None,
                                          operation="median",
                                          **kwargs):
    """
    Calculates the geomedian or geomedoid using a multi-band processing method.
    Parameters
    ----------
    dataset_in: xarray.Dataset
        A dataset retrieved from the Data Cube; should contain:
        coordinates: time, latitude, longitude (in that order)
        variables: variables to be mosaicked (e.g. red, green, and blue bands)
    clean_mask: np.ndarray
        An ndarray of the same shape as `dataset_in` - specifying which values to mask out.
        If no clean mask is specified, then all values are kept during compositing.
    no_data: int or float
        The no data value.
    dtype: str or numpy.dtype
        A string denoting a Python datatype name (e.g. int, float) or a NumPy dtype (e.g.
        np.int16, np.float32) to convert the data to.
    operation: str in ['median', 'medoid']
    Returns
    -------
    dataset_out: xarray.Dataset
        Composited data with the format:
        coordinates: y, x
        variables: same as dataset_in
    """
    # Default to masking nothing.
    if clean_mask is None:
        clean_mask = create_default_clean_mask(dataset_in)
    assert operation in ['median', 'medoid'], "Only median and medoid operations are supported."

    # Save dtypes because masking with Dataset.where() converts to float64.
    band_list = list(dataset_in.data_vars)
    dataset_in_dtypes = {}
    for band in band_list:
        dataset_in_dtypes[band] = dataset_in[band].dtype
    # Mask out clouds and scan lines.
    dataset_in = dataset_in.where((dataset_in != no_data) & clean_mask)

    arrays = [dataset_in[band] for band in band_list]
    stacked_data = np.stack(arrays)
    #changed around as other
    bands_shape, time_slices_shape, lon_shape, lat_shape = stacked_data.shape[0], \
                                                           stacked_data.shape[1], stacked_data.shape[2], \
                                                           stacked_data.shape[3]
    # Reshape to remove lat/lon
    reshaped_stack = stacked_data.reshape(bands_shape, time_slices_shape,
                                          lat_shape * lon_shape)
    # Build zeroes array across time slices.
    hdmedians_result = np.zeros((bands_shape, lat_shape * lon_shape))

    # For each pixel (lat/lon combination), find the geomedian or geomedoid across time.
    for x in range(reshaped_stack.shape[2]):
        try:
            hdmedians_result[:, x] = hd.nangeomedian(
                reshaped_stack[:, :, x], axis=1) if operation == "median" else hd.nanmedoid(
                reshaped_stack[:, :, x], axis=1)
        except ValueError as e:
            # If all bands have nan values across time, the geomedians are nans.
            hdmedians_result[:, x] = np.full((bands_shape), np.nan)
    output_dict = {
        value: (('y', 'x'), hdmedians_result[index, :].reshape(lat_shape, lon_shape))
        for index, value in enumerate(band_list)
    }
    dataset_out = xr.Dataset(output_dict,
                             coords={'y': dataset_in['y'],
                                     'x': dataset_in['x']})
    #dataset_out = restore_or_convert_dtypes(dtype, band_list, dataset_in_dtypes, dataset_out, no_data)
    return dataset_out    

In [24]:
from dask.diagnostics import ProgressBar
with ProgressBar():
    output = create_hdmedians_multiple_band_mosaic(DS)

<ipython-input-23-a957f5d9d870>:45: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  clean_mask = np.ones(dataset_in[first_data_var].shape).astype(np.bool)


[########################################] | 100% Completed |  1hr 50min 17.0s
[########################################] | 100% Completed |  1hr 30min 10.0s


MemoryError: Unable to allocate 80.8 GiB for an array with shape (40, 16470, 16470) and data type float64